# Hessian-free optimization
Attempt to implement HF optimization per []

## Initial setup

In [37]:
import tensorflow as tf
import numpy as np

## Case 0: quadratic function
First, we will compute the Hessian for the quadratic function:

$f(x) = x^T A x + b^T x + c$

$\nabla f = A x + b$

$H(f) = A$

(assuming A is symmetric)

Let's create some naive functions to start:

In [7]:
def naive_grad(f, x):
    return tf.gradients(f, x)

def naive_hessian(f, x):
    return tf.gradients(naive_grad(f, x), x)

In [8]:
tf.reset_default_graph()

# Build graph
x = tf.placeholder(tf.float32, shape=[4, 1], name='x')
A = tf.constant(np.array([[1, 2, 3, 4], # sum = 10
                          [2, 1, 5, 6], # sum = 14
                          [3, 5, 1, 7], # sum = 16
                          [4, 6, 7, 1]]), # sum = 18
                dtype=tf.float32)
b = tf.constant(np.array([[1], 
                          [2],
                          [3],
                          [4]]), 
                dtype=tf.float32)
c = tf.constant(np.array([1]), dtype=tf.float32)
f = 0.5 * tf.transpose(x) @ (A @ x) + tf.transpose(b) @ x + c
#f = 0.5 * tf.matmul(tf.transpose(x), tf.matmul(A, x)) + tf.matmul(tf.transpose(b), x) + c

In [9]:
sess = tf.Session()
x_ = np.ones([4, 1])
feed_dict={x: x_}
print("f(x)")
print(sess.run(f, feed_dict=feed_dict))
print("\ndfdx (auto)")
print(sess.run(naive_grad(f, x), feed_dict=feed_dict))
print("\ndfdx (analytic)")
print(sess.run(A @ x + b, feed_dict=feed_dict))
print("\ndf2dx2 (auto)")
print(sess.run(naive_hessian(f, x), feed_dict=feed_dict))
print("\ndf2dx2 (analytic)")
print(sess.run(A, feed_dict=feed_dict))

f(x)
[[40.]]

dfdx (auto)
[array([[11.],
       [16.],
       [19.],
       [22.]], dtype=float32)]

dfdx (analytic)
[[11.]
 [16.]
 [19.]
 [22.]]

df2dx2 (auto)
[array([[10.],
       [14.],
       [16.],
       [18.]], dtype=float32)]

df2dx2 (analytic)
[[1. 2. 3. 4.]
 [2. 1. 5. 6.]
 [3. 5. 1. 7.]
 [4. 6. 7. 1.]]


The autobackprop algorithm in tensorflow returns the summation over axis 1 for dy/dx, which is exactly what occurs when A is summed over axis 1. Per the tf docs:

> Constructs symbolic derivatives of sum of ys w.r.t. x in xs.

> ys and xs are each a Tensor or a list of tensors. grad_ys is a list of Tensor, holding the gradients received by the ys. The list must be the same length as ys.

> gradients() adds ops to the graph to output the derivatives of ys with respect to xs. It returns a list of Tensor of length len(xs) where each tensor is the sum(dy/dx) for y in ys.

So we must slice over each component of x (that is, treat them as separate `x` in `xs`), and then stack the results back together to get the Hessian. The opposite case would occur for the Jacobian: slice over each component of `y` to get dy_i/dx, and then stack the results together.

Our gradient function, however, seems okay, let's rename it for simplicity.

In [10]:
grad = naive_grad

def true_hessian(f, x):
    df2dx2 = []
    
    # Return list of components in x
    #x = tf.slice(x, [0, 0], [x.get_shape()[0], 1]) # not iterable
    #x = tf.unstack(x, axis=0)
    # This doesn't work because it creates a side branch that f does
    # not depend on, thus returning a gradient of [None]
    
    # Get gradients, which returns sum(df_i/dx)
    dfdx = grad(f, x)[0]
    
    # Get number of components in x
    dfdx_dim = dfdx.get_shape().as_list()
    dfdx_size = 1
    for d in dfdx_dim:
        dfdx_size *= d
    
    # Iterate over each component in dfdx
    for i in range(dfdx_size):
        begin = [i // dfdx_dim[1], i % dfdx_dim[1]]
        #print(begin)
        dfdx_i = tf.slice(dfdx, begin, [1, 1]) # [1, 1]
        #print(dfdx_i)
        df2dx_i2 = tf.gradients(dfdx_i, x)[0] # [[df/dx_idx1, df/dx_idx2, ...], [...]]
        df2dx_i2 = tf.reshape(df2dx_i2, [1, -1]) # [df/dx_idx1, df/dx_idx2, ...]
        df2dx2.append(df2dx_i2)
    
    hessian = tf.concat(df2dx2, axis=0) # [[[df/dx_1dx_1, df/dx_1dx_2, ...],
                                        #  [df/dx_2dx_1, df/dx_2dx_2, ...],
                                        #  ...                            ]]
    return hessian

In [11]:
print("\ndf2dx2 (corrected)")
print(sess.run(true_hessian(f, x), feed_dict=feed_dict))


df2dx2 (corrected)
[[1. 2. 3. 4.]
 [2. 1. 5. 6.]
 [3. 5. 1. 7.]
 [4. 6. 7. 1.]]


Great, it works! We can now rename our hessian function as well.

In [12]:
hessian = true_hessian

## Case 1: Two-layer MLP
Let's extrapolate our findings to a simple multi-layer perceptron.

In [13]:
tf.reset_default_graph()

# Build graph
x = tf.placeholder(tf.float32, shape=[3, 1], name='x')
W1 = tf.constant(np.array([[1, 2, 3],
                           [4, 5, 6],
                           [7, 8, 9]]) / 10.0,
                 dtype=tf.float32)
b1 = tf.constant(np.ones([3, 1]), 
                 dtype=tf.float32)
h1 = W1 @ x + b1
z1 = tf.sigmoid(h1)

W2 = tf.constant(np.array([[1, 4, 7],
                           [2, 5, 8],
                           [3, 6, 9]]) / 10.0,
                 dtype=tf.float32)
b2 = tf.constant(np.ones([3, 1]), 
                 dtype=tf.float32)
h2 = W2 @ z1 + b2
z2 = tf.sigmoid(h2)
y = tf.reduce_sum(z2)

# Analytic gradients
dydz2 = tf.ones(tf.shape(z2))
dydh2 = z2 * (1.0 - z2) * dydz2
dydz1 = tf.transpose(W2) @ dydh2
dydh1 = z1 * (1.0 - z1) * dydz1
dydx = tf.transpose(W1) @ dydh1

In [14]:
sess = tf.Session()
x_ = np.ones([3, 1])
feed_dict = {x: x_}

print("y")
print(sess.run(y, feed_dict=feed_dict))

print("\ndydx (auto)")
print(sess.run(grad(y, x), feed_dict=feed_dict))

print("\ndydx (analytic)")
print(sess.run(dydx, feed_dict=feed_dict))

y
2.7463741

dydx (auto)
[array([[0.00771682],
       [0.00966905],
       [0.01162129]], dtype=float32)]

dydx (analytic)
[[0.00771682]
 [0.00966905]
 [0.01162129]]


Remember that gradient in the auto-backpropagation returns the sum of all components of $f$ wrt each $x_i$:

`tf.gradients(f, x)` --> $
\begin{bmatrix}
\sum_i \frac{\partial f_i}{\partial x_1} \\ 
\sum_i \frac{\partial f_i}{\partial x_2} \\ 
... \\
\sum_i \frac{\partial f_i}{\partial x_n}
\end{bmatrix}$

whereas the Jacobian of $f$ wrt $x$, $J(f, x)$, is traditionally written as:

$J(f, x) = \begin{bmatrix}
\frac{\partial f_1}{\partial x_1} & \frac{\partial f_1}{\partial x_2} & ... & \frac{\partial f_1}{\partial x_n}\\ 
... \\
\frac{\partial f_m}{\partial x_1} & \frac{\partial f_m}{\partial x_2} & ... & \frac{\partial f_m}{\partial x_n}\\ 
\end{bmatrix}$

So in order to sum over the Jacobian properly to get the equivalent of `tf.gradients`, we must transpose the Jacobian and then sum over the axis 1, which effectively sums over $f_i$ for each $x_i$.

In [15]:
def jacobian(f, x):
    # Need to iterate over f_i, analogous to iterating over x_i for hessian (?)
    J = []
    for f_i in tf.unstack(f, axis=0):
        df_idx = grad(f_i, x)[0] # [[df_idx_1], [df_idx_2], ...]
        df_idx = tf.transpose(df_idx) # [df_idx_1, df_idx_2, ...]
        J.append(df_idx)
        #print(df_idx.get_shape().as_list())
    
    J = tf.stack(J, axis=0) # [[[df_1dx_1, df_1dx_2, ...]],
                            #  [[df_2dx_2, df_2dx_2, ...]]
                            #                        ...]]]
    
    return tf.squeeze(J) # [m, 1, n] --> [m, n]

In [16]:
print("dh2dz1 (auto)")
print(sess.run(grad(h2, z1), feed_dict=feed_dict)[0])
print()

print("dh2dz1 (analytic, Jacobian)")
dh2dz1 = W2
print(sess.run(dh2dz1, feed_dict=feed_dict))
print()

print("dh2dz1 (analytic, accumulated)")
print(np.sum(sess.run(dh2dz1, feed_dict=feed_dict), axis=0, keepdims=True).T)
print()

print("dh2dz1 (auto, Jacobian)")
print(sess.run(jacobian(h2, z1), feed_dict=feed_dict))

dh2dz1 (auto)
[[0.6]
 [1.5]
 [2.4]]

dh2dz1 (analytic, Jacobian)
[[0.1 0.4 0.7]
 [0.2 0.5 0.8]
 [0.3 0.6 0.9]]

dh2dz1 (analytic, accumulated)
[[0.6]
 [1.5]
 [2.4]]

dh2dz1 (auto, Jacobian)
[[0.1 0.4 0.7]
 [0.2 0.5 0.8]
 [0.3 0.6 0.9]]


Calculating the Hessian of even a simple 3-layer MLP is not trivial to do by hand. Consider the derivative of $y$ wrt two components of the input, $x_i$ and $x_j$:

$\begin{aligned}
\frac{\partial^2 y}{\partial x_i \partial x_j} = H_{ij}
&= \frac{\partial}{\partial x_j} \left (\frac{\partial y}{\partial x_i} \right ) \\
&= \frac{\partial}{\partial x_j} \left ( \sum_k \frac{\partial y}{\partial h_k} \frac{\partial h_k}{\partial x_i} \right ) \\
&= \sum_k \left [ \frac{\partial}{\partial x_j} \left ( \frac{\partial y}{\partial h_k} \frac{\partial h_k}{\partial x_i} \right ) \right ] \\
&= \sum_k \left [ \frac{\partial}{\partial x_j} \left ( \frac{\partial y}{\partial h_k} \right ) \frac{\partial h_k}{\partial x_i} + \frac{\partial y}{\partial h_k} \frac{\partial}{\partial x_j} \left ( \frac{\partial h_k}{\partial x_i} \right ) \right ] \\
&= \sum_k \left [ \sum_l \left ( \frac{\partial^2 y}{\partial h_k \partial h_l} \frac{\partial h_l}{\partial x_j} \right ) \frac{\partial h_k}{\partial x_i} + \frac{\partial y}{\partial h_k} \frac{\partial^2 h_k}{\partial x_i \partial x_j} \right ] \\
&= \sum_k \sum_l \left ( \frac{\partial^2 y}{\partial h_k \partial h_l} \frac{\partial h_l}{\partial x_j} \frac{\partial h_k}{\partial x_i} \right ) + \sum_k \left ( \frac{\partial y}{\partial h_k} \frac{\partial^2 h_k}{\partial x_i \partial x_j} \right )
\end{aligned}$

Clearly this scales quickly. We will manually implement the Hessian calculation per [Bishop, 1992]. Note that running the graph multiple times is not the most efficient implementation, but it does allow for the most transparency when calculating (and debugging) our manual implementation.

In [21]:
np.set_printoptions(precision=3)

# Automatic Hessian calculation
print("H (auto)")
print(sess.run(hessian(y, W1), feed_dict=feed_dict))
#print(sess.run(tf.hessians(y, W1), feed_dict=feed_dict))
print()

# Manual Hessian calculation
# Get unit activations (z)
x_ = np.ones([3, 1])
feed_dict = {x: x_}
x_size = x.get_shape().as_list()[0]
h1_size = h1.get_shape().as_list()[0]
h2_size = h2.get_shape().as_list()[0]
num_units = x_size + h1_size + h2_size
z_ = np.concatenate(sess.run([x, z1, z2], feed_dict=feed_dict))

# Get first derivatives of activations wrt affine transformations
dzdh_ = np.ones([num_units])
dz2dh2 = z2 * (1.0 - z2)
dz1dh1 = z1 * (1.0 - z1)
dzdh_[x_size:] = np.squeeze(np.concatenate(sess.run([dz1dh1, dz2dh2], feed_dict=feed_dict)))
#print("dzdh")
#print(dzdh_)
#print()

# Get second derivatives of activations wrt affine transformations
d2zdh2_ = np.zeros([num_units])
d2z2dh22 = z2 * (1 - z2) * (1 - 2*z2)
d2z1dh12 = z1 * (1 - z1) * (1 - 2*z1)
d2zdh2_[x_size:] = np.squeeze(np.concatenate(sess.run([d2z1dh12, d2z2dh22], feed_dict=feed_dict)))
#print("d2zdh2")
#print(d2zdh2_)
#print()

# Create global matrix W_ that holds all parameters from all W st
# W_[i, j] refers to the weight from unit j to unit i
W_ = np.zeros([x_size + h1_size + h2_size, x_size + h1_size + h2_size])
W_[x_size:x_size + h1_size, :x_size] = sess.run(W1)
W_[x_size + h1_size:, x_size:x_size + h1_size] = sess.run(W2)
#print("W_init")
#print(W_)
#print()

# g_li represents da_l/da_i and is found by forward propagation. Thus, the initial conditions
# set values for da_i/da_i = 1 and begin at the first unit.
g_ = np.zeros([num_units, num_units])
g_[np.arange(len(g_)), np.arange(len(g_))] = 1.0
#print("g_init")
#print(g_)
#print()
for i in range(x_size + h1_size + h2_size):
    for j in range(x_size + h1_size + h2_size):
        #for k in range(x_size + h1_size + h2_size):
        #    g_[j, i] += dzdh_[k] * W_[j, k] * g_[k, i]
        g_[j, i] += np.sum(dzdh_ * W_[j, :] * g_[:, i])

#print("g")
#print(g_)
#print()

# o_n represents dy/da_n and is found by backpropagation. Thus, the initial conditions
# set values for the output units m and begin at the output units.
o_ = np.zeros([num_units])
o_[-h2_size:] = dzdh_[-h2_size:] * 1.0
for i in range(num_units)[::-1]:
    for j in range(num_units)[::-1]:
        o_[i] += dzdh_[i] * W_[j, i] * o_[j]

#print("o")
#print(o_)
#print()

# b_ni represents do_n/da_i and is also found by backpropagation, via applying the 
# product rule after substituting for o_n. The initial conditions thus also set values 
# for the output units.
b_ = np.zeros([num_units, num_units])
b_[-h2_size:, x_size:] = d2zdh2_[-h2_size:, np.newaxis] * 1.0 * g_[-h2_size:, x_size:]
i_range = range(num_units)
for i in i_range[::-1]:
    if i in i_range[:x_size]:
        n_range = i_range[:]
    elif i in i_range[:x_size + h1_size]:
        n_range = i_range[x_size:]
    else:
        n_range = i_range[x_size + h1_size:]
    
    for n in n_range[::-1]:
        for r in range(num_units)[::-1]:
            b_[n, i] += d2zdh2_[n] * g_[n, i] * W_[r, n] * o_[r] + dzdh_[n] * W_[r, n] * b_[r, i]
#print("b")
#print(b_)
#print()

# Finally, the Hessian is calculated using the equation:
#     
#    H(W_ij, W_nl) = z_j * o_n * dzdh_l * g_li + z_j * z_l * b_ni
#    
# where i, j, n, l are indices of the global weight matrix.
H_ = np.zeros([W_.size, W_.size])
for r in range(W_.size):
    for s in range(W_.size):
        i = r // W_.shape[1]
        j = r % W_.shape[1]
        n = s // W_.shape[1]
        l = s % W_.shape[1]
        H_[r, s] = z_[j] * o_[n] * dzdh_[l] * g_[l, i] + z_[j] * z_[l] * b_[n, i]

print("H (manual)")
print(H_)
print()

H (auto)
[[-0.004 -0.004 -0.004 -0.    -0.    -0.    -0.    -0.    -0.   ]
 [-0.004 -0.004 -0.004 -0.    -0.    -0.    -0.    -0.    -0.   ]
 [-0.004 -0.004 -0.004 -0.    -0.    -0.    -0.    -0.    -0.   ]
 [-0.    -0.    -0.    -0.007 -0.007 -0.007 -0.    -0.    -0.   ]
 [-0.    -0.    -0.    -0.007 -0.007 -0.007 -0.    -0.    -0.   ]
 [-0.    -0.    -0.    -0.007 -0.007 -0.007 -0.    -0.    -0.   ]
 [-0.    -0.    -0.    -0.    -0.    -0.    -0.005 -0.005 -0.005]
 [-0.    -0.    -0.    -0.    -0.    -0.    -0.005 -0.005 -0.005]
 [-0.    -0.    -0.    -0.    -0.    -0.    -0.005 -0.005 -0.005]]

H (manual)
[[ 0.004 -0.004 -0.004 ...  0.     0.     0.   ]
 [ 0.004 -0.004 -0.004 ...  0.     0.     0.   ]
 [ 0.004 -0.004 -0.004 ...  0.     0.     0.   ]
 ...
 [ 0.     0.     0.    ... -0.042 -0.043 -0.041]
 [ 0.     0.     0.    ... -0.043 -0.044 -0.042]
 [ 0.     0.     0.    ... -0.044 -0.045 -0.043]]



While our automatic hessian function (using tensorflow) shapes and slices parameters specific to the query, our manual method calculates the Hessian of a global matrix that includes interactions amongst all parameters. To find the Hessian of the parameters in `W_in`, for example, we must slice our total Hessian matrix accordingly.

In [22]:
def _get_W_idx(in_idx, out_idx, num_units):
    i, j = np.meshgrid(out_idx * num_units, in_idx, indexing='ij')
    W_idx = (i + j).flatten()
    return W_idx

def get_W_idx(layer_sizes):
    # Calculate total number of parameters
    num_params = 0
    for size in layer_sizes:
        num_params += size
    
    # Slice parameters at cutoffs corresponding to unrolled weight matrices
    idx = [] # each component represents indices of weight matrix
    param = 0 # current param
    params = np.arange(num_params)
    for i in range(1, len(layer_sizes)):
        idx1 = params[param:param + layer_sizes[i-1]] # unit indices of current layer
        param += layer_sizes[i-1]
        idx2 = params[param:param + layer_sizes[i]] # unit indices of next layer
        idx.append(_get_W_idx(idx1, idx2, num_params)) # params connecting current to next layer
    
    return idx

In [30]:
idx = get_W_idx([x_size, h1_size, h2_size]) # param indices
idx = [np.meshgrid(idx_, idx_, indexing='ij') for idx_ in idx] # slices
H_W1 = H_[idx[0]]
print(H_W1)

[[-0.004 -0.004 -0.004 -0.    -0.    -0.    -0.    -0.    -0.   ]
 [-0.004 -0.004 -0.004 -0.    -0.    -0.    -0.    -0.    -0.   ]
 [-0.004 -0.004 -0.004 -0.    -0.    -0.    -0.    -0.    -0.   ]
 [-0.    -0.    -0.    -0.007 -0.007 -0.007 -0.    -0.    -0.   ]
 [-0.    -0.    -0.    -0.007 -0.007 -0.007 -0.    -0.    -0.   ]
 [-0.    -0.    -0.    -0.007 -0.007 -0.007 -0.    -0.    -0.   ]
 [-0.    -0.    -0.    -0.    -0.    -0.    -0.005 -0.005 -0.005]
 [-0.    -0.    -0.    -0.    -0.    -0.    -0.005 -0.005 -0.005]
 [-0.    -0.    -0.    -0.    -0.    -0.    -0.005 -0.005 -0.005]]


In [31]:
# Compare all matrices
print("H_W1 (auto)")
print(sess.run(hessian(y, W1), feed_dict=feed_dict))
print()

print("H_W1 (manual)")
print(H_[idx[0]])
print()

print("H_W2 (auto)")
print(sess.run(hessian(y, W2), feed_dict=feed_dict))
print()

print("H_W2 (manual)")
print(H_[idx[1]])
print()

H_W1 (auto)
[[-0.004 -0.004 -0.004 -0.    -0.    -0.    -0.    -0.    -0.   ]
 [-0.004 -0.004 -0.004 -0.    -0.    -0.    -0.    -0.    -0.   ]
 [-0.004 -0.004 -0.004 -0.    -0.    -0.    -0.    -0.    -0.   ]
 [-0.    -0.    -0.    -0.007 -0.007 -0.007 -0.    -0.    -0.   ]
 [-0.    -0.    -0.    -0.007 -0.007 -0.007 -0.    -0.    -0.   ]
 [-0.    -0.    -0.    -0.007 -0.007 -0.007 -0.    -0.    -0.   ]
 [-0.    -0.    -0.    -0.    -0.    -0.    -0.005 -0.005 -0.005]
 [-0.    -0.    -0.    -0.    -0.    -0.    -0.005 -0.005 -0.005]
 [-0.    -0.    -0.    -0.    -0.    -0.    -0.005 -0.005 -0.005]]

H_W1 (manual)
[[-0.004 -0.004 -0.004 -0.    -0.    -0.    -0.    -0.    -0.   ]
 [-0.004 -0.004 -0.004 -0.    -0.    -0.    -0.    -0.    -0.   ]
 [-0.004 -0.004 -0.004 -0.    -0.    -0.    -0.    -0.    -0.   ]
 [-0.    -0.    -0.    -0.007 -0.007 -0.007 -0.    -0.    -0.   ]
 [-0.    -0.    -0.    -0.007 -0.007 -0.007 -0.    -0.    -0.   ]
 [-0.    -0.    -0.    -0.007 -0.007 -0.007 -0. 

They match up!

## Case 2: Static RNN
Let's now attempt to further extend our findings to a simple recurrent neural network.

### First-order gradients

In [33]:
class StaticRNN:
    def __init__(self, trace_length, x_size, h_size, y_size, seed=1234):
        tf.reset_default_graph()
        tf.set_random_seed(1234)

        # Build new graph with single W variable
        self.x = tf.placeholder(tf.float32, shape=[trace_length, 1, x_size], name='x')
        
        with tf.name_scope("params"):
            # Input weights
            self.W1 = tf.random_normal([x_size, h_size], dtype=tf.float32, name='W1')
            self.b1 = tf.ones([1, h_size], dtype=tf.float32, name='b1')

            # Recurrent weights
            self.W2 = tf.random_normal([h_size, h_size], dtype=tf.float32, name='W2')
            self.b2 = tf.ones([1, h_size], dtype=tf.float32, name='b2')

            # Ouput weights
            self.W3 = tf.random_normal([h_size, y_size], dtype=tf.float32, name='W3')
            self.b3 = tf.random_normal([1, y_size], dtype=tf.float32, name='b3')
            
            # Vectors containing all parameters
            self.W = tf.Variable(tf.concat([tf.reshape(self.W1, [-1]), 
                                            tf.reshape(self.W2, [-1]),
                                            tf.reshape(self.W3, [-1])],
                                           axis=0), name='W')
            self.b = tf.Variable(tf.concat([tf.reshape(self.b1, [-1]),
                                            tf.reshape(self.b2, [-1]),
                                            tf.reshape(self.b3, [-1])],
                                           axis=0), name='b')

            # Get layer variables from concatenated vectors
            self.W_in = tf.reshape(self.W[:x_size*h_size], [x_size, h_size], name='W_in')
            self.b_in = tf.reshape(self.b[:x_size], [1, x_size], name='b_in')
            self.W_rec = tf.reshape(self.W[x_size*h_size:x_size*h_size + h_size**2], 
                                    [h_size, h_size], name='W_rec')
            self.b_rec = tf.reshape(self.b[x_size:x_size + h_size], [1, h_size], name='b_rec')
            self.W_out = tf.reshape(self.W[-h_size*y_size:], [h_size, y_size], name='W_out')
            self.b_out = tf.reshape(self.b[-y_size:], [1, y_size], name='b_out')

        # Initialize time series [x_t, h_t, y_t]
        xs = tf.unstack(self.x, axis=0, name='xs')
        self.h_init = tf.zeros([1, h_size], name='h_init')
        z = []
        h = [self.h_init]
        y = []
        
        # Create unrolled hidden layer for each time step
        for i, x_t in enumerate(xs):
            with tf.name_scope("hidden_%d" % (i+1)):
                z_tx = x_t @ self.W_in
                z_th = h[-1] @ self.W_rec
                z_t = tf.add_n([z_tx, z_th, self.b_rec], name='z_%d' % (i+1))
                h_t = tf.tanh(z_t, name='h_%d' % (i+1))
            
            with tf.name_scope("y_%d" % (i+1)):
                y_t = tf.add(h_t @ self.W_out, self.b_out, name='y_%d' % (i+1))
            
            z.append(z_t)
            h.append(h_t)
            y.append(y_t)
        
        # Save lists of time series values
        self.z = z
        self.h = h[1:]
        self.y = y
        self.J = tf.reduce_sum(self.y, name='J')

        # Save dimensions
        self.trace_length = trace_length # unrolled time dimension
        self.x_size = x_size
        self.h_size = h_size
        self.y_size = y_size
    
    @property
    def auto_grads(self):
        y = self.J
        xs = [self.W_in, self.W_rec, self.W_out]
        return [tf.gradients(y, x) for x in xs]
    
    @property
    def manual_grads(self):
        dJdW_in = [] # dJdW_in = Σ(dJ/dy * dy/dh * dh/dz * dz/dW_in) for [0, t]
        dJdW_rec = [] # dJdW_rec = Σ(dJ/dy * dy/dh * dh/dz * dz/dW_rec) for [0, t]
        dJdW_out = [] # dJdW_out = Σ(dJ/dy * dy/dW_out) for [0, t]

        for t in range(self.trace_length)[::-1]: # Σ(dJ/dq * ... * dq/dW) for t' in [0, t]
            # Chain rule: dJ_t/dz_j * dz_j/dW_in = dJ_t/dz_t * Σ(dz_t/dz_j) * dz_j/dW_in
            dJ_tdW_in = []
            for j in range(t+1)[::-1]: # Σ(dz_t/dz_j) for j in [0, t]
                # Compute dJ_t/dz_t
                dJ_tdy_t = tf.ones([y_size, 1])
                dy_tdh_t = self._dydh()
                dh_tdz_j = self._dhdz(self.h[t])
                
                # Compute dz_t/dz_j = Π(dz_k/dz_(k-1)) for k in [j, t]
                for k in range(j, t)[::-1]:
                    dh_tdz_j = (self._dhdz(self.h[k]) @ self._dzdh()) @ dh_tdz_j
                
                # Compute dz_j/dW_in
                dz_jdW_in = self._dzdW_in(j)
                
                # Add dJ_t/dz_j * dz_j/dW_in
                dJ_tdW_in.append((dz_jdW_in @ (dh_tdz_j @ (dy_tdh_t @ dJ_tdy_t))))
            
            # Chain rule: dJ_t/dz_j * dz_j/dW_rec = dJ_t/dz_t * Σ(dz_t/dz_j) * dz_j/dW_rec
            dJ_tdW_rec = []
            for j in range(t+1)[::-1]: # Σ(dz_t/dz_j) for j in [0, t-1]
                # Compute dJ_t/dz_t
                dJ_tdy_t = tf.ones([y_size, 1])
                dy_tdh_t = self._dydh()
                dh_tdz_j = self._dhdz(self.h[t])
                
                # Compute dz_t/dz_j = Π(dz_k/dz_(k-1)) for k in [j, t-1]
                for k in range(j, t)[::-1]:
                    dh_tdz_j = (self._dhdz(self.h[k]) @ self._dzdh()) @ dh_tdz_j
                
                # Compute dz_j/dW_rec
                dz_jdW_rec = self._dzdW_rec(j)
                
                # Add dJ_t/dz_j * dz_j/dW_rec
                dJ_tdW_rec.append((dz_jdW_rec @ (dh_tdz_j @ (dy_tdh_t @ dJ_tdy_t))))
            
            # dJ_tdW_out = dJ/dy * dy/dW_out
            dJ_tdy_t = tf.ones([y_size, 1])
            dy_tdW_out = self._dydW_out(t)
            dJ_tdW_out = (dy_tdW_out @ dJ_tdy_t)
            
            # Append to lists of dJ_tdW for all t
            dJdW_in.append(tf.add_n(dJ_tdW_in)) # sum over j
            dJdW_rec.append(tf.add_n(dJ_tdW_rec)) # sum over j
            dJdW_out.append(dJ_tdW_out)
        
        # Sum over all t
        dJdW_in = tf.add_n(dJdW_in)
        dJdW_rec = tf.add_n(dJdW_rec)
        dJdW_out = tf.add_n(dJdW_out)
        
        # Reshape from vectorized to matrix form 
        # Transpose needed due to reshaping order (rows, then columns)
        dJdW_in = tf.reshape(dJdW_in, [self.h_size, self.x_size]) # reverse order
        dJdW_in = tf.transpose(dJdW_in)
        dJdW_rec = tf.reshape(dJdW_rec, [self.h_size, self.h_size]) # reverse order
        dJdW_rec = tf.transpose(dJdW_rec)
        dJdW_out = tf.reshape(dJdW_out, [self.y_size, self.h_size]) # reverse order
        dJdW_out = tf.transpose(dJdW_out)
        
        return [dJdW_in, dJdW_rec, dJdW_out]
    
    def _dydh(self):
        return self.W_out

    def _dhdz(self, h):
        return tf.eye(self.h_size) * (1.0 - h**2)

    def _dzdh(self):
        return self.W_rec

    def _dzdx(self):
        return self.W_in

    
    # All _dzdW must unroll W to a vector [W11, ..., W1n, W21, ..., W2n, ..., Wmn]
    # in order to compute the Jacobian [[dz1dW11, ..., dz1dWmn],
    #                                   [dz2dW11, ..., dz2dWmn],
    #                                   ...
    #                                   [dzndW11, ..., dzndWmn]]
    # The functions below actually return the transpose of the Jacobian for ease
    # of calculation.
    def _dzdW_in(self, t):
        # Item assignment not supported in tf (unlike numpy) :-(
        #dW_in = tf.zeros([self.h_size, self.x_size, self.h_size])
        #dW_in[tf.range(self.h_size), :, tf.range(h_size)] = self.x[t] # not supported in tf
        # So instead must use (loop + stack) x 2
        dW_in = []
        for i in range(self.h_size):
            dW_in_i = []
            for j in range(self.h_size):
                if i == j:
                    dW_in_i.append(tf.ones([self.x_size]) * tf.squeeze(self.x[t]))
                else:
                    dW_in_i.append(tf.zeros([self.x_size]))
            dW_in.append(tf.stack(dW_in_i, axis=1))
        dW_in = tf.stack(dW_in, axis=0)
        #dW_in = tf.transpose(dW_in, [0, 2, 1])
        return tf.reshape(dW_in, [-1, self.h_size])

    def _dzdW_rec(self, t):
        # Same indexing problem as explained in _dzdW_in
        #dW_rec = tf.zeros([self.h_size, self.h_size, self.h_size])
        #dW_rec[tf.range(self.h_size), :, tf.range(self.h_size)] = self.h[t-1]
        dW_rec = []
        if t > 0:
            h_prev = self.h[t-1]
        else:
            h_prev = self.h_init
        for i in range(self.h_size):
            dW_rec_i = []
            for j in range(self.h_size):
                if i == j:
                    dW_rec_i.append(tf.ones([self.h_size]) * tf.squeeze(h_prev))
                else:
                    dW_rec_i.append(tf.zeros([self.h_size]))
            dW_rec.append(tf.stack(dW_rec_i, axis=1))
        dW_rec = tf.stack(dW_rec, axis=0)
        #dW_rec = tf.transpose(dW_rec, [0, 2, 1])
        return tf.reshape(dW_rec, [-1, self.h_size])

    def _dydW_out(self, t):
        # Same indexing problem as explained in _dzdW_in
        #dW_out = tf.zeros([self.y_size, self.h_size, self.y_size])
        #dW_out[tf.range(self.y_size), :, tf.range(self.y_size)] = self.h[t]
        dW_out = []
        for i in range(self.y_size):
            dW_out_i = []
            for j in range(self.y_size):
                if i == j:
                    dW_out_i.append(tf.ones([self.h_size]) * tf.squeeze(self.h[t]))
                else:
                    dW_out_i.append(tf.zeros([self.h_size]))
            dW_out.append(tf.stack(dW_out_i, axis=1))
        dW_out = tf.stack(dW_out, axis=0)
        #dW_out = tf.transpose(dW_out, [0, 2, 1])
        return tf.reshape(dW_out, [-1, self.y_size])

In [35]:
# Set network settings
trace_length = 3
x_size = 3
h_size = 3
y_size = 2
seed = 1234
rnn = StaticRNN(trace_length, x_size, h_size, y_size, seed=seed)

# Initialize tf session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Create (deterministic) input state
r = np.random.RandomState(seed)
x_ = r.randn(trace_length, 1, x_size)
feed_dict = {rnn.x: x_}

# Get intermediate and output values
h_, y_, J_ = sess.run([rnn.h, rnn.y, rnn.J], feed_dict=feed_dict)

print("x")
print(x_)
print()

print("h")
for h_i in h_:
    print(h_i)
print()

print("y")
for y_i in y_:
    print(y_i)
print()

print("J")
print(J_)

x
[[[ 0.471 -1.191  1.433]]

 [[-0.313 -0.721  0.887]]

 [[ 0.86  -0.637  0.016]]]

h
[[ 0.978 -0.826 -0.953]]
[[ 0.98  -0.722 -0.278]]
[[ 0.966  0.991 -0.003]]

y
[[1.433 2.553]]
[[1.731 1.62 ]]
[[ 0.16  -3.197]]

J
4.299705


In [36]:
# Calculate gradients automatically
auto_grads = sess.run(rnn.auto_grads, feed_dict=feed_dict)
for g in auto_grads:
    print(g[0])
print()

# Calculate gradients manually
manual_grads = sess.run(rnn.manual_grads, feed_dict=feed_dict)
for g in manual_grads:
    print(g)

[[ 0.124  0.116 -0.207]
 [-0.227  2.051  0.911]
 [ 0.17  -2.449 -0.821]]
[[ 0.22  -1.653 -1.011]
 [-0.171  1.389  0.813]
 [-0.12   1.564  0.724]]
[[ 2.925  2.925]
 [-0.558 -0.558]
 [-1.233 -1.233]]

[[ 0.124  0.116 -0.207]
 [-0.227  2.051  0.911]
 [ 0.17  -2.449 -0.821]]
[[ 0.22  -1.653 -1.011]
 [-0.171  1.389  0.813]
 [-0.12   1.564  0.724]]
[[ 2.925  2.925]
 [-0.558 -0.558]
 [-1.233 -1.233]]


They match!

### Second-order gradients

In [39]:
from copy import copy
np.set_printoptions(precision=4)

# Auto hessian
d2JdW_in2 = true_hessian(rnn.J, rnn.W_in)
H_auto = sess.run(d2JdW_in2, feed_dict=feed_dict)
print("auto hessian:")
print(H_auto)
print()

# Numerical estimation (brute force)
feed_dict = {rnn.x: x_}
dJdW_in = sess.run(rnn.auto_grads[0][0], feed_dict=feed_dict).flatten()
N = len(dJdW_in)
H_num = np.zeros([N, N])
epsilon = 5e-4
W_in = sess.run(rnn.W_in)
W_in_shape = W_in.shape
W_in = W_in.flatten()
for j in range(N):
    W_in_ = copy(W_in)
    delta = epsilon * W_in_[j]
    W_in_[j] += delta
    W_in_ = np.reshape(W_in_, W_in_shape)
    feed_dict = {rnn.x: x_, rnn.W_in: W_in_}
    dJdW_in_ = sess.run(rnn.auto_grads[0][0], feed_dict=feed_dict).flatten()
    H_num[:, j] = (dJdW_in_ - dJdW_in) / delta
print("numerical hessian:")
print(H_num)
print()

# This is pretty difficult due to the balance between precision of estimate (small ε)
# and numerical stability (large ε)
is_close = np.allclose(H_auto, H_num)
print("Auto hessian {}verified".format("" if is_close else "NOT "))

auto hessian:
[[-2.4052e-01 -1.3991e-02 -6.8192e-03  1.8270e-01  3.9125e-03 -2.0616e-02
  -4.5936e-02 -3.4976e-03  2.5305e-02]
 [-1.3991e-02 -5.3603e-01 -6.9091e-03 -1.3071e-02  1.5965e-01 -1.4832e-01
   3.1889e-02 -2.6690e-01  1.8075e-01]
 [-6.8192e-03 -6.9091e-03 -9.8862e-02  5.6398e-03  6.3765e-02  1.1305e-01
  -1.3975e-03 -6.0784e-02 -1.3318e-01]
 [ 1.8270e-01 -1.3071e-02  5.6398e-03 -3.8489e-01  2.2344e-03  1.6539e-02
   3.3489e-01 -1.8561e-03 -2.0296e-02]
 [ 3.9125e-03  1.5965e-01  6.3765e-02  2.2344e-03 -2.2147e+00  1.7141e-02
   7.9934e-03  2.7224e+00 -2.2553e-02]
 [-2.0616e-02 -1.4832e-01  1.1305e-01  1.6539e-02  1.7141e-02 -6.7106e-01
   7.1318e-06  2.6350e-02  8.1085e-01]
 [-4.5936e-02  3.1889e-02 -1.3975e-03  3.3489e-01  7.9934e-03  7.1320e-06
  -4.0060e-01 -1.1342e-02  2.1263e-05]
 [-3.4976e-03 -2.6690e-01 -6.0784e-02 -1.8561e-03  2.7224e+00  2.6350e-02
  -1.1342e-02 -3.2687e+00 -3.0556e-02]
 [ 2.5305e-02  1.8075e-01 -1.3318e-01 -2.0296e-02 -2.2553e-02  8.1085e-01
   2.126

## Hessian-free optimization

### Conjugate gradients

In [41]:
class StaticRNN_HF(StaticRNN):
    def __init__(self, trace_length, x_size, h_size, y_size, seed=1234):
        # Initialize base class
        StaticRNN.__init__(self, trace_length, x_size, h_size, y_size, seed=seed)
        
        # Build loss function
        self.y_target = tf.placeholder(tf.float32, 
                                       shape=[trace_length, y_size],
                                       name='y_target')
        with tf.name_scope("loss"):
            self.loss = tf.reduce_mean(tf.square(self.y - self.y_target), name="mse_loss")
    
        # Build auto gradients
        with tf.name_scope("gradients"):
            self.grads = tf.gradients(self.loss, [self.W, self.b])          

In [42]:
rnn = StaticRNN_HF(trace_length, x_size, h_size, y_size)

Let's figure out how to compute the matrix-vector products $G_f v$, where $G_f$ is the Gauss-Newton matrix and $v$ is the direction. The Gauss-Newton matrix for a single training example is $G_f = J_{s,\theta}^{T} (L \circ g)'' J_{s,\theta} \big|_{\theta = \theta_n}$. First, we will compute $J_{s,\theta} v$ using the method in [Pearlmutter, 1994].

## Archive

### Old gradient code
Outside StaticRNN class, so get to use numpy :-)

Using derivations from this [blog post](http://willwolf.io/2016/10/18/recurrent-neural-network-gradients-and-lessons-learned-therein/).

In [567]:
W_in_ = sess.run(rnn.W_in)
W_rec_ = sess.run(rnn.W_rec)
W_out_ = sess.run(rnn.W_out)

def dydh():
    return W_out_

def dhdz(h):
    return np.eye(h_size) * (1.0 - h**2)

def dzdh():
    return W_rec_

def dzdx():
    return W_in_

def dzdW_in(t):
    # Need to vectorize W_in in order for matmul to work in dhdz @ dzdW_in
    dW_in = np.zeros([h_size, x_size, h_size])
    dW_in[np.arange(h_size), :, np.arange(h_size)] = x_[t]
    #dW_in = np.transpose(dW_in, [0, 2, 1])
    return dW_in.reshape([h_size, -1])

def dzdW_rec(t):
    # Need to vectorize W_in in order for matmul to work in dhdz @ dzdW_in
    dW_rec = np.zeros([h_size, h_size, h_size])
    dW_rec[np.arange(h_size), :, np.arange(h_size)] = h_[t-1]
    #dW_rec = np.transpose(dW_rec, [0, 2, 1])
    return dW_rec.reshape([h_size, -1])

def dydW_out(t):
    dW_out = np.zeros([y_size, h_size, y_size])
    dW_out[np.arange(y_size), :, np.arange(y_size)] = h_[t]
    #dW_out = np.transpose(dW_out, [0, 2, 1])
    return dW_out.reshape([y_size, -1])

In [568]:
dJdW_in = []
dJdW_rec = []
dJdW_out = []

for t in range(trace_length)[::-1]:
    # Compute dJdW_in = Σ(dJdy * dydh * dhdz * dzdW_in) for [0, t]
    dJ_tdW_in = 0
    for j in range(t+1)[::-1]:
        dJ_tdy_t = np.ones([1, y_size])
        dy_tdh_t = dydh()
        #dh_tdz_j = np.ones([1, h_size])
        dh_tdz_j = dhdz(h_[j])
        for k in range(j)[::-1]:
            #print(dhdz(h_[k]) )
            dh_tdz_j *= (dhdz(h_[k]) @ dzdh())
        dz_jdW_in = dzdW_in(j)
        #print(dJ_tdy_t.shape, dy_tdh_t.shape, dh_tdz_j.shape, dz_jdW_in.shape)
        dJ_tdW_in += ((dJ_tdy_t @ dy_tdh_t.T) @ dh_tdz_j.T) @ dz_jdW_in
    dJdW_in.append(dJ_tdW_in)
    
    # Compute dydW_rec = Σ(dydh * dhdz * dzdW_rec) for [0, t]
    dJ_tdW_rec = 0
    for j in range(t)[::-1]:
        dJ_tdy_t = np.ones([1, y_size])
        dy_tdh_t = dydh()
        dh_tdz_j = np.ones([1, h_size])
        for k in range(j)[::-1]:
            dh_tdz_j *= (dhdz(h_[k]) @ dzdh())
        dz_jdW_rec = dzdW_rec(j)
        dJ_tdW_rec += ((dJ_tdy_t @ dy_tdh_t.T) @ dh_tdz_j.T) @ dz_jdW_rec
    dJdW_rec.append(dJ_tdW_rec)
    
    # Compute dJdW_in = Σ(dJdy * dydh * dhdz * dzdW_in) for [0, t]
    dJ_tdW_out = 0
    for j in range(t+1)[::-1]:
        dJ_tdy_t = np.ones([1, y_size])
        dy_tdW_out = dydW_out(j)
        dJ_tdW_out += dJ_tdy_t @ dy_tdW_out
    dJdW_out.append(dJ_tdW_out)

print(dJdW_in[0].reshape([x_size, h_size]))
# throws error with trace_length=1 because inner loop not accessed
#print(dJdW_rec[0].reshape([h_size, h_size]))
print(dJdW_out[0].reshape([h_size, y_size]))

[[ 0.043 -0.546 -0.017]
 [-0.108  1.379  0.042]
 [ 0.129 -1.659 -0.051]]
[[ 0.978  0.978]
 [-0.826 -0.826]
 [-0.953 -0.953]]


[[ 0.008  0.037  0.059]
 [-0.021 -0.094 -0.148]
 [ 0.026  0.113  0.179]]
[[0.906 0.906]
 [0.918 0.918]
 [0.928 0.928]]
